# Introduction To Python, using functions and libraries
by Martín Araya  
  
# functions

we can use **our own previously defined functions** as well as **publictly available functions**

- read imput data from include files using a function previously defined

In [ ]:
from MyFunctions import readKeyword

In [ ]:
# use the readKeyword function to read the data from the include files
IncludesFolder = './sample_data/'
rawPermX = readKeyword( IncludesFolder + 'PERMX.GRDECL' )

In [ ]:
# this function returns a dictionary with all the keywords found in the file with a list of the values for that each keyword
# check the type of variable returned
print( 'type of PermX:' , type(rawPermX) )

In [ ]:
# check for the keys of the dictionary
print( 'keys of PermX:' , rawPermX.keys() )

In [ ]:
# check what is inside
print( 'type of object inside the dictionary:' , type( rawPermX['PERMX'] ))

In [ ]:
# check the type of elements of the list
print( 'type of items inside the list in the dictionary:' , type( rawPermX['PERMX'][0] ))

In [ ]:
print( 'sample of items in the imported lists:')
print( rawPermX['PERMX'][:10] )
# notice that items in the list are 'string' type
# notice also that some items represent repeated values, like '13*0' 

In [ ]:
# use the readKeyword function to read the data from the include files
rawPermZ = readKeyword( IncludesFolder + 'PERMZ.GRDECL' )
rawPoro = readKeyword( IncludesFolder + 'PORO.GRDECL' )
rawRockType = readKeyword( IncludesFolder + 'SATNUM.GRDECL' )
rawNTG = readKeyword( IncludesFolder + 'NTG.GRDECL' )
rawACTNUM = readKeyword( IncludesFolder + 'ACTNUM.GRDECL' )

In [ ]:
### check the what we have imported
# check the length of our data 
print( 'length of the lists:' , len(rawPermX['PERMX']) , len(rawPermZ['PERMZ']) , len(rawPoro['PORO']) , len(rawRockType['SATNUM']) ) 
# different sizes!!!

In [ ]:
# we can use a function already refined to convert this repeated values to their regular representation:
#   ['5*4'] = ['4', '4', '4', '4', '4']
from MyFunctions import expandKeyword
expandedPermX = expandKeyword( rawPermX['PERMX'] )
expandedPermZ = expandKeyword( rawPermZ['PERMZ'] )
expandedPoro = expandKeyword( rawPoro['PORO'] )
expandedRockType = expandKeyword( rawRockType[ list(rawRockType.keys())[0] ] )
expandedNTG = expandKeyword( rawNTG['NTG'] )
expandedACTNUM = expandKeyword( rawACTNUM['ACTNUM'] )

In [ ]:
# check the expanded list
print( 'sample of items in the imported lists after expansion:')
print( expandedPermX[:19] )

In [ ]:
# check the length of our data 
print( 'length of the lists:' , len(expandedPermX) , len(expandedPermZ) , len(expandedPoro) , len(expandedRockType) ) 
# same size

# working with lists of values (data)
## the loop way

because our data is composed by string type items, we have to comvert it in order to make calculations and, on top of that, we can't opperate directly with the list.

In [ ]:
# will use datetime to measure how long it takes to run
import datetime as dt

- very slow option, not recommended for big arrays/matrix:

In [ ]:
SlowStart = dt.datetime.now()

# create empty lists to load the data to cast loaded PermX values from string to float
PermX , PermZ , Poro = [] , [] , []
for i in range( len(expandedPermX) ) :
    PermX.append( float(expandedPermX[i]) )
    PermZ.append( float(expandedPermZ[i]) )
    Poro.append( float(expandedPoro[i]) )
    
SlowEnd = dt.datetime.now()
print( 'Slow method elapsed time:' , SlowEnd - SlowStart )

we will calculate KvKh using the lists of data readed

In [ ]:
# calculate KvKh in new list
KvKh = []
for i in range( len(expandedPermX) ) :
    KvKh.append( ( float(expandedPermZ[i]) / PermX[i] ) if PermX[i] > 0 else None )

- better option to work with lists

In [ ]:
QuickStart = dt.datetime.now()

PermX , PermZ , Poro = [None]*len(expandedPermX) , [None]*len(expandedPermX) , [None]*len(expandedPermX)
for i in range( len(expandedPermX) ) :
    PermX[i] = float(expandedPermX[i])
    PermZ[i] = float(expandedPermZ[i])
    Poro[i] = float(expandedPoro[i])
    
QuickEnd = dt.datetime.now()
print( 'Quick method elapsed time:' , QuickEnd - QuickStart )
print( 'Quick method is' , round( ( ( ( SlowEnd - SlowStart ) - ( QuickEnd - QuickStart ) ) / ( SlowEnd - SlowStart ) )*100 , 0 ) , '% faster than Slow method' )

## unsing numpy

In [ ]:
import numpy as np

NumpyStart = dt.datetime.now()

PermX = np.array( expandedPermX , dtype='float' )
PermZ = np.array( expandedPermZ , dtype='float' )
Poro = np.array( expandedPoro , dtype='float' )

NumpyEnd = dt.datetime.now()
print( 'NUmpy method elapsed time:' , NumpyEnd - NumpyStart )
print( 'Numpy method is' , round( ( ( ( SlowEnd - SlowStart ) - ( NumpyEnd - NumpyStart ) ) / ( NumpyEnd - NumpyStart ) )*100 , 0 ) , '% faster than Slow method' )

In [ ]:
# using numpy arrays makes this operations easy
# now creating Kv/Kh is simply a division, no loops involved
KvKh = None
KvKh = PermZ / PermX

# a warning may be printed in case some division by zero occurs
# the division is done for the pair of numbers where PermX is >0
KvKh[:30]

In [ ]:
# casting the remaining data
RockType = np.array( expandedRockType , dtype='int' )
NTG = np.array( expandedNTG , dtype='float' )
ACTNUM = np.array( expandedACTNUM , dtype='int')

# reviewing the data
  
After importing your data is good practice to check your data:
- check data type is correct
- make histograms
- check correlations, seaborn.pairplot will be useful
- check statistics, pandas.describe becomes useful

In [ ]:
# import matplotlib.pyplot to make some charts
import matplotlib.pyplot as plt

In [ ]:
# make histograms using hist function from matplotlib
PxHist = plt.figure(figsize=(10,10))
plt.hist(PermX,bins=[1,10,100,1000,10000])
plt.xscale('log')
plt.title('PermX')
plt.show()

In [ ]:
# crossplot of vertical and horizontal permeability
KZXScat = plt.figure(figsize=(10,10))
plt.scatter(y=PermZ,x=PermX)
plt.ylabel('PermZ')
plt.xlabel('PermX')
plt.show()

In [ ]:
# Poro-Perm crossplot
PoroPerm = plt.figure(figsize=(10,10))
plt.scatter(y=PermX,x=Poro)
plt.ylabel('PermX')
plt.yscale('log')
plt.xlabel('Poro')
plt.xlim(0,0.3)
plt.ylim(0.1,10000)
plt.show()

## putting filters to the data and color to the plots

it could be convenient to **filter** values of non-significative data, like:  
- inactive cells
- undefined values
- might be outliers
- values out of the group or range of interest  
  
Filters in numpy are boolean arrays (True and False) of the same size and shape of the array to be filtered.
  
Filters can be generated on the fly using the same or other array tested by a condition operator

In [ ]:
# prepare a filter using ACTNUM, NTG, PORO and PermX all of them > 0
ActiveCells = (ACTNUM>0) & (NTG>0) & (Poro>0) & (PermX>0)
ActiveCells[:50]

matplotlib **colors** can be set using the names of the predefined colors or identifiying the color by its RGB code as a tuple.  
  
for further details on the predefined colors please read:
https://matplotlib.org/3.3.2/gallery/color/named_colors.html

In [ ]:
# we can prepare an array of colors to be applied to the data points, based on the RockTypes
print( 'RockType values range:\n  min:' , min(RockType) , 'max:' , max(RockType) ) # check range of RockType values

In [ ]:
# prepare an empty array of strings
Colors = np.array(['']*len(RockType),dtype='<U8')

In [ ]:
# using filters to set the color names according to the RockType values
Colors[RockType==1] = 'yellow' 
Colors[RockType==2] = 'orange'
Colors[RockType==3] = 'red'
Colors[RockType==4] = 'blue'

In [ ]:
# now keep only the ActiveCels for PermX, Poro and Colors before plotting (handy because will be used several times)
filteredColors = Colors[ActiveCells]
filteredPermX = PermX[ActiveCells]
filteredPoro = Poro[ActiveCells]

In [ ]:
# poro-perm chart colored by facies

# using transparency on the points gives the chart a representation of the density of the values
# the more solid the color appears the more probable this value is

PoroPerm = plt.figure(figsize=(12,12))
plt.scatter(y=filteredPermX,x=filteredPoro,color=filteredColors,marker='o',s=1,alpha=0.03)
plt.ylabel('PermX')
plt.yscale('log')
plt.xlabel('Poro')
plt.ylim(1,10000)
plt.xlim(0.05,0.25)
plt.grid(which='major',color='gray',alpha=0.75)
plt.grid(which='minor',color='gray',alpha=0.25)
plt.show()

## using pandas
  
The good thing of pandas is that it can allocate data of different types and that the columns retain their label.  
Also, pandas comes with a lot of handy functions to operate, analyze and plot the data.

In [ ]:
# using Pandas could make this task easier
import pandas as pd

In [ ]:
# prepare a dictionary with the names of columns the dataframe will have and the type of data of each column to cast into it
columnTypes = {'PermX':'float',
               'PermZ':'float',
               'Poro':'float',
               'RockType':'int',
               'NTG':'float',
               'ACTNUM':'int'}

In [ ]:
# prepare a dictionary with the names of the columns as keys and the list of values for each column
data = {'PermX':expandedPermX,
        'PermZ':expandedPermZ,
        'Poro':expandedPoro,
        'RockType':expandedRockType,
        'NTG':expandedNTG,
        'ACTNUM':expandedACTNUM}

In [ ]:
Props = pd.DataFrame(data).astype(columnTypes)

In [ ]:
# create the calculated variables

# mathematical operations
Props['KvKh'] = Props['PermZ'] / Props['PermX']
Props['RQI'] = 0.0314 * ( Props['PermX'] / Props['Poro'] ) ** 0.5

# logical operations
Props['ActiveCells'] = (Props['ACTNUM']>0) & (Props['NTG']>0) & (Props['Poro']>0) & (Props['PermX']>0)

# using filters and list comprehension
Props['Colors'] = [ 'yellow' if RT==1 else 'orange' if RT==2 else 'red' if RT==3 else 'blue' for RT in Props['RockType'] ]

In [ ]:
# take a filtered dataframe, with only data for the ActiveCells
filteredProps = Props[Props.ActiveCells==True]

### pandas is deeply integrated with matplotlib 
We can use matplotlib functions directly from the dataframe object
Some plots like **boxplot** and **histogram** are directly integrated into pandas.  
Other plots can be invoqued using function .plot and then the matplotlib function, like **.plot.scatter**

In [ ]:
# pandas is deeply integrated with matplotlib 
# we can use matplotlib functions directly from the dataframe object
filteredProps[['Poro','RockType']].boxplot(by='RockType' , figsize=(12,7) )

**return_type='both'** paramenter is requiered to be able to manipulate the figure using matplotlib after pandas has created it.

In [ ]:
# we can decorate the boxplots as much as we want

# set colors
RTcolors = ['yellow','orange','red','blue']

# create the plot and save it in a variable
boxplt = filteredProps[['Poro','RockType']].boxplot(by='RockType' , figsize=(12,7) , grid=False , patch_artist = True,\
                                                 return_type='both' ) 

# manipulate the boxplot using matplotlib
for row_key, (ax,row) in boxplt.iteritems():
    for i,box in enumerate(row['boxes']):
        box.set_facecolor(RTcolors[i])    

plt.show()

In [ ]:
filteredProps[['RockType']].hist(figsize=(12,7))

In [ ]:
filteredProps.plot.scatter(x='Poro',y='RQI',figsize=(12,7))

### several dimensions can be represented in a 2D plot
taking advantange of other attributes, like:
- size of the data point
- color of the data point
- alpha (transparency) of the data point
  
Also useful to consider that, in a cloud of data, the transparency of the data points helps visualize the density of the data

In [ ]:
# in this Poro-Perm plot, 
#   the color indicated the RockType
#   the size of the point is proportional to the RQI

PoroPermRQI = plt.figure(figsize=(12,12))
plt.scatter(y=filteredProps.PermX,x=filteredProps.Poro,color=filteredColors,marker='o',s=filteredProps.RQI**5,alpha=0.05)
plt.ylabel('PermX')
plt.yscale('log')
plt.xlabel('Poro')
plt.ylim(1,10000)
plt.xlim(0.05,0.25)
plt.grid(which='major',color='gray',alpha=0.75)
plt.grid(which='minor',color='gray',alpha=0.25)
plt.show()

# using seaborn to quickly discover the data

**seaborn** works on top of **matplotlib** and provides a lot of useful functions to easly create diagnostic chats

In [ ]:
# importing seaborn library
import seaborn as sns

### seaborn.pairplot

**pairplot** creates a matrix of charts with cross-plot and distribution plots for every pair of columns in the dataset.  
  
- It is convenient to define a new figure before calling seaborn plots. Otherwise, the plot will added to the last figure object created.  

To create a pairplot:  
- simply pass as first argument the dataframe with the columns desired to be in the plot.
- if the data has labels (discreate values that identifies each data point), it can be passed using **hue** argument

In [ ]:
# define a new figure before calling seaborn plots
plt.figure(figsize=(12,12))

# pass the dataframe with the selected columns and the hue argument with the name of the column that has the labels
sns.pairplot( filteredProps[['PermX','PermZ','Poro','RockType']] , hue='RockType' )
plt.show()

### seaborn.regplot

**regplot** creates a cross-plot with correlation and uncertainty range on top of the data points.  
The arguments **x** and **y** of regplot can be *numpy array* or *pandas series*

Taking advantage that the seaborn plots are drawn on the last figure object created, several *points & regressions* can be shown together. 
- Simply use a loop or several lines of code one after the other to call the regplot function for each set of data-points.  
  
To change the properties of the points in the chart, use the argument **scatter_kws** to pass a dictionary of the matplotlib parameters you would like to change, i.e.:  
- scatter_kws = {'alpha':0.50 , 's':1}
- to change color **use *'color'*** keyword because regplot already uses *'color'* and then the keyword *'c'* will have conflicts

In [ ]:
# create a new figure 
plt.figure(figsize=(12,12))

# call regplot for each RockType separately using a loop
for i in range( 1 , max(filteredProps.RockType)+1 ) :
    sns.regplot( x=filteredProps['Poro'][filteredProps.RockType==i] , \
                 y=np.log(filteredProps['PermX'][filteredProps.RockType==i]) , \
                 scatter_kws={'alpha': 0.05,'s':1} )

In this dataset PermZ has been created as a constant fraction of PermX.  
  
To better illustrate the regplot functionality, we need to have more random relationship between PermZ and PermX.  
In order to do that we can create a random multiplier to later apply to PermZ using **numpy.random**  

In [ ]:
# generate an array of random values normaly distributed arround 1 with standard deviation of 0.1 
# The length of the array must match the length of the PermZ array
noise = abs( np.random.normal(1, 0.75, len(Props.PermZ) ) )

# notice that
print(" len(Props.PermZ) :", len(Props.PermZ) , \
      "\n len(filteredProps.PermZ):", len(filteredProps.PermZ) )

In [ ]:
# review the 'noise' we have created
plt.hist( noise )

Remmember that we should use the length of the **original dataframe**
- *len(Props.PermZ)* 873300   
  
and not the length of the **filtered dataframe**  
- *len(filteredProps.PermZ)* 482912   

becasuse the **filtered dataframe is just a *view* of the original dataframe**, not an actual dataframe. We should operate on the original data and then regenerate the view.  

It is possible to operate on the view, but a warning will be rised from pandas.

In [ ]:
# create a nee column in the dataframe, applying the 'noise' to 'PermZ'
Props['noisyPermZ'] = Props['PermZ'] * noise

In [ ]:
# print the columns available in filteredProps
filteredProps.columns
# regenerate the view of the dataframe
filteredProps = Props[Props.ActiveCells==True]
# print again the columns available in filteredProps
filteredProps.columns

In [ ]:
# or check if the new column is present in the view
'noisyPermZ' in filteredProps.columns

Now we can do the pairplot again, using _noisyPermZ_ instead of _PermZ_:

In [ ]:
# define a new figure before calling seaborn plots
plt.figure(figsize=(12,12))

# pass the dataframe with the selected columns and the hue argument with the name of the column that has the labels
sns.pairplot( filteredProps[['PermX','noisyPermZ','Poro','RockType']] , hue='RockType' )
plt.show()

An finally create a regression plot, regplot, to illustrate in a log-log chart the relationship between _PermX_ and _noisyPermZ_.  
In the next plot we will see:
- the data points colored in red
- the linear regression as a black line
- the confidence interval as a shadow centered in the regresion line

In [ ]:
# create a new figure
plt.figure(figsize=(12,12))

# generate x-plot of PermX and noisyPermZ only for RockType 3
sns.regplot( x=filteredProps['PermX'][filteredProps['RockType']==3] , \
             y=filteredProps['noisyPermZ'][filteredProps['RockType']==3] , \
             scatter_kws = {'alpha':0.01 , 's':3 , 'color':'red'} , color='black' )

# using matplotlib we can change the axis scale
plt.yscale('log')
plt.xscale('log')
plt.xlim((5,5000))
plt.ylim((0.5,500))
plt.show()